# Crop REMA stripes by the polygon from 3 (around channel)

based on https://rasterio.readthedocs.io/en/stable/topics/masking-by-shapefile.html 
but also using tarfile

This script unzips the raster, crops it, then saves a tiff

In [1]:
# !conda activate arran_gis

In [2]:
# !jupyter kernelspec list --json

In [1]:
import rasterio as rio
import rasterio.mask
import tarfile
import fiona

import os
import sys
#working_dir = '/Volumes/arc_02/REMOTE_SENSING/'
#working_dir = "/Users/home/whitefar/DATA/Channel/line_channel_traced_for_50kms/"
working_dir = '/Users/home/whitefar/DATA/FIELD_ANT_19/'

sys.path.append(os.path.abspath(working_dir))
sys.path.append(os.path.abspath('/Users/home/whitefar/DATA/code/'))
#sys.path.append(os.path.abspath('/Users/home/whitefar/DATA/Channel/line_channel_traced_for_50kms/'))
os.chdir(working_dir)

In [2]:
# !which python

In [11]:
#zipped_stripes_path = "/Users/home/whitefar/DATA/Channel/line_channel_traced_for_50kms/test_stripe/"
zipped_stripes_path = "/Volumes/arc_02/REMOTE_SENSING/REMA/REMA_stripes_over_kamb_channel/"

#cropped_stripes_path = zipped_stripes_path
#cropped_stripes_path = "/Users/home/whitefar/DATA/Channel/line_channel_traced_for_50kms/cropped_stripes/"
cropped_stripes_path = "/Volumes/arc_02/REMOTE_SENSING/REMA/REMA_stripes_1920_fieldsite/"


zipped_stripes_fnames = [f for f in os.listdir(zipped_stripes_path) if os.path.isfile(os.path.join(zipped_stripes_path, f))]

temp_tiffs_path = "/Users/home/whitefar/DATA/Channel/line_channel_traced_for_50kms/stripe_tiffs/" #save them temporararily

tiffs_path

zipped_stripes_fnames

['SETSM_WV01_20151102_10200100437EF000_10200100477F7400_seg1_2m_v1.0.tar.gz',
 'SETSM_WV01_20150104_1020010038253500_1020010038CD0C00_seg1_2m_v1.0.tar.gz',
 'SETSM_WV01_20131222_10200100274B7500_102001002981EC00_seg1_2m_v1.0.tar.gz',
 'SETSM_WV02_20131117_103001002A8C0D00_1030010029C81600_seg1_2m_v1.0.tar.gz',
 'SETSM_WV02_20161219_103001006116CC00_10300100638F8E00_seg1_2m_v1.0.tar.gz',
 'SETSM_WV01_20161219_102001005ADD9F00_102001005A4F0700_seg1_2m_v1.0.tar.gz',
 'SETSM_WV01_20161027_1020010056BA2E00_1020010058DFD800_seg1_2m_v1.0.tar.gz',
 'SETSM_WV01_20141209_1020010037BD1F00_10200100387ABC00_seg3_2m_v1.0.tar.gz',
 'SETSM_WV01_20161003_1020010057910400_102001005845EA00_seg1_2m_v1.0.tar.gz',
 'SETSM_WV01_20141106_1020010035707400_102001003747DB00_seg1_2m_v1.0.tar.gz',
 'SETSM_WV01_20141118_10200100376E3200_1020010036DE5C00_seg1_2m_v1.0.tar.gz',
 'SETSM_WV02_20121224_103001001D86CA00_103001001D886000_seg1_2m_v1.0.tar.gz',
 'SETSM_WV02_20151010_103001004B706800_103001004A9F3700_seg1_2m_

In [12]:
# !python -m ipykernel install --user --name arran_gis

In [47]:
#with fiona.open("/Users/home/whitefar/DATA/Channel/line_channel_traced_for_50kms/channel_area.shp", "r") as shapefile:
#    channel_area = [feature["geometry"] for feature in shapefile]
with fiona.open("/Users/home/whitefar/DATA/FIELD_ANT_19/study_area_buffer.shp", "r") as shapefile:
    channel_area = [feature["geometry"] for feature in shapefile]
with fiona.open("/Users/home/whitefar/DATA/Channel/line_channel_traced_for_50kms/polygons_stripes_50kms_up_channel.shp", "r") as shapefile:
    stripe_polygons = [feature["geometry"] for feature in shapefile]

In [65]:
from shapely.geometry import Polygon

stripes = [Polygon(stripe_polygons[i]["coordinates"][0]) for stripe in enumerate(stripe_polygons)]

field_area_buffer = Polygon(channel_area[0]["coordinates"][0])

have_intersection = [stripe.intersects(field_area_buffer) for stripe in stripes]

have_intersection


[False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False]

In [14]:
def crop_stripe(zipped_stripe_fname, zipped_stripes_path, cropped_stripes_path, temp_tiffs_path):
       
    #remove .tar.gz from the name
    stripe_name = zipped_stripe_fname[:-7]
    
    zipped_stripe_path =  zipped_stripes_path + zipped_stripe_fname 
    
    #path for output tiff
    tiff_stripe_fname = stripe_name + "_dem.tif"
    
    #extract the .tar.gz and save as .tiff
    with tarfile.open(zipped_stripe_path) as tar:
        stripe_tiff = tar.extract(member=tiff_stripe_fname, path=temp_tiffs_path)
        
    #crop the .tiff    
    with rio.open(temp_tiffs_path + tiff_stripe_fname) as src:
        #print(src.transform)
        out_image, out_transform = rasterio.mask.mask(src, channel_area,crop=True)
        data_type = out_image.dtype
        out_meta = src.meta.copy()
        
    #print(out_transform)
    
    out_meta.update({"driver": "GTiff",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform,
                 "dtype" : data_type})
    
    cropped_stripe_path = cropped_stripes_path + tiff_stripe_fname
    
    #write the cropped tiff to file
    with rio.open(cropped_stripe_path, "w", **out_meta) as dest:
        dest.write(out_image)
    
    #remove the uncropped tiff
    os.remove(temp_tiffs_path + tiff_stripe_fname)
        
    print(stripe_name + "cropped and written to "+ cropped_stripe_path)

In [17]:
for i, zipped_stripe_fname in enumerate(zipped_stripes_fnames):
    
    crop_stripe(zipped_stripe_fname, zipped_stripes_path, cropped_stripes_path, temp_tiffs_path)
    
    ValueError: Input shapes do not overlap raster.
    
    print(f"{i+1}/{len(zipped_stripes_fnames)}")
       

ValueError: Input shapes do not overlap raster.